In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('lstm.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnt Even See Comeys Let...,Darrell Lucus,House Dem Aide: We Didnt Even See Comeys Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [4]:
df.isna().sum()

id         0
title      0
author    93
text       0
label      1
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
X=df.drop('label',axis=1)

In [8]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didnt Even See Comeys Let...,Darrell Lucus,House Dem Aide: We Didnt Even See Comeys Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...


In [9]:
y=df['label']

In [10]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: object

In [11]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
from tensorflow.keras.preprocessing.text import one_hot

In [15]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.stem import PorterStemmer

In [17]:
messages=X.copy()

In [18]:
messages.reset_index(inplace=True)

In [19]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didnt Even See Comeys Let...,Darrell Lucus,House Dem Aide: We Didnt Even See Comeys Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...


In [20]:
corpus=[]

In [21]:
ps=PorterStemmer()
for i in range(len(messages)):
    sent=re.sub('[^a-zA-Z]'," ",messages['title'][i])
    sent=sent.lower()
    words=sent.split()
    sent=[ps.stem(word) for word in words if word not in nltk.corpus.stopwords.words('english')]
    sent=' '.join(sent)
    corpus.append(sent)

In [22]:
Voc_size=5000
onehot_repr=[one_hot(words,Voc_size) for words in corpus]

In [23]:
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=30)

In [24]:
import numpy as np

In [25]:
np.array(embedded_docs)

array([[4717, 2186, 1234, ...,    0,    0,    0],
       [1740, 3871, 3118, ...,    0,    0,    0],
       [2182, 4877,  919, ...,    0,    0,    0],
       ...,
       [2637, 2368,  297, ...,    0,    0,    0],
       [4777, 3233, 4938, ...,    0,    0,    0],
       [4201, 2095,  183, ...,    0,    0,    0]])

In [26]:
X_fi=np.array(embedded_docs)
y_fi=np.array(y)

In [27]:
from sklearn.model_selection import train_test_split as tts

In [28]:
Xtr,Xte,ytr,yte=tts(X_fi,y_fi,test_size=0.2,random_state=42)

In [37]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=40))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(Xtr,ytr,validation_data=(Xte,yte),epochs=10,batch_size=64) 

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node compile_loss/binary_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ASUS\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\ASUS\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\ASUS\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\Users\ASUS\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\ASUS\anaconda3\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\Users\ASUS\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\ASUS\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\ASUS\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users